In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(temperature=0.1)

memory = ConversationBufferMemory(
    return_messages=True, memory_key="chat_history"
)


cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/homework4.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a helpful assistant. Answer questions using only the"
                " following context. If you don't know the answer just say you"
                " don't know, don't make it up:\n\n{context}"
            ),
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        (
            "system",
            "You can also respond based on the information stored in memory.",
        ),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | RunnablePassthrough.assign(chat_history=load_memory)
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(memory.load_memory_variables({}))
    return result.content


result = invoke_chain("Is Aaronson guilty?")
print(result)

{'chat_history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')]}
Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.


In [138]:
invoke_chain("What message did he write in the table?")

{'chat_history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='He wrote "2+2=5" in the dust on the table.')]}


'He wrote "2+2=5" in the dust on the table.'

In [139]:
invoke_chain("Who is Julia?")

{'chat_history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='He wrote "2+2=5" in the dust on the table.'), HumanMessage(content='Who is Julia?'), AIMessage(content='Julia is a character who was involved with the protagonist in the story.')]}


'Julia is a character who was involved with the protagonist in the story.'

In [143]:
invoke_chain("What was the first question I asked?")

{'chat_history': [HumanMessage(content='Is Aaronson guilty?'), AIMessage(content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'), HumanMessage(content='What message did he write in the table?'), AIMessage(content='He wrote "2+2=5" in the dust on the table.'), HumanMessage(content='Who is Julia?'), AIMessage(content='Julia is a character who was involved with the protagonist in the story.'), HumanMessage(content='What was the first question I asked?'), AIMessage(content="I don't know, I can only provide information from the text."), HumanMessage(content='What was the first question I asked?'), AIMessage(content="I don't know, I can only provide information from the text."), HumanMessage(content='What was the first answer I asked?'), AIMessage(content="You asked about Aaronson's guilt."), HumanMessage(content='What was the first question I asked?'), AIMessage(content='You asked if Aaronson was guilty.')]}


'You asked if Aaronson was guilty.'